In [1]:
import argparse
import scanpy as sc
import os
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np

In [2]:
adata = sc.read('/nfs/turbo/umms-indikar/shared/projects/DARPA_AI/in-silico-reprogramming/unperturbed/fibroblast.h5ad')  # Adjust as necessary

In [3]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

X = adata.X
scaler = StandardScaler(with_mean=False)
X = scaler.fit_transform(X)

In [6]:
n_clusters = 10

In [11]:
kmeans = KMeans(n_clusters=n_clusters, n_init=1, random_state=42, verbose=1, max_iter=1)

In [18]:
cluster_labels = kmeans.fit_predict(X)

Initialization complete
Iteration 0, inertia 2629916928.0.


In [17]:
len(cluster_labels)

38151

In [12]:
help(kmeans.fit_predict)

Help on method fit_predict in module sklearn.cluster._kmeans:

fit_predict(X, y=None, sample_weight=None) method of sklearn.cluster._kmeans.KMeans instance
    Compute cluster centers and predict cluster index for each sample.
    
    Convenience method; equivalent to calling fit(X) followed by
    predict(X).
    
    Parameters
    ----------
    X : {array-like, sparse matrix} of shape (n_samples, n_features)
        New data to transform.
    
    y : Ignored
        Not used, present here for API consistency by convention.
    
    sample_weight : array-like of shape (n_samples,), default=None
        The weights for each observation in X. If None, all observations
        are assigned equal weight.
    
    Returns
    -------
    labels : ndarray of shape (n_samples,)
        Index of the cluster each sample belongs to.



In [ ]:
adata.obs[f'kmeans_{args.n_clusters}_{args.run}'] = cluster_labels

In [4]:
help(KMeans)

Help on class KMeans in module sklearn.cluster._kmeans:

class KMeans(_BaseKMeans)
 |  KMeans(n_clusters=8, *, init='k-means++', n_init='warn', max_iter=300, tol=0.0001, verbose=0, random_state=None, copy_x=True, algorithm='lloyd')
 |  
 |  K-Means clustering.
 |  
 |  Read more in the :ref:`User Guide <k_means>`.
 |  
 |  Parameters
 |  ----------
 |  
 |  n_clusters : int, default=8
 |      The number of clusters to form as well as the number of
 |      centroids to generate.
 |  
 |  init : {'k-means++', 'random'}, callable or array-like of shape             (n_clusters, n_features), default='k-means++'
 |      Method for initialization:
 |  
 |      'k-means++' : selects initial cluster centroids using sampling based on
 |      an empirical probability distribution of the points' contribution to the
 |      overall inertia. This technique speeds up convergence. The algorithm
 |      implemented is "greedy k-means++". It differs from the vanilla k-means++
 |      by making several t

In [ ]:
import argparse
import scanpy as sc
import os
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np

# Set up argument parsing
parser = argparse.ArgumentParser()
parser.add_argument('--n_clusters', type=int, required=True, help='Number of clusters')
parser.add_argument('--run', type=int, required=True, help='Run number')
args = parser.parse_args()

# Load your AnnData object
# Replace with your actual code to load the data
adata = sc.read('/nfs/turbo/umms-indikar/shared/projects/DARPA_AI/in-silico-reprogramming/unperturbed/fibroblast.h5ad')  # Adjust as necessary

# Normalize and log-transform the data
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# Extract the count matrix and standardize the features
X = adata.X
scaler = StandardScaler(with_mean=False)
X_scaled = scaler.fit_transform(X)

# Perform K-means clustering
kmeans = KMeans(n_clusters=args.n_clusters, init='k-means++', n_init=1, random_state=42 + args.run)
cluster_labels = kmeans.fit_predict(X_scaled)
adata.obs[f'kmeans_{args.n_clusters}_{args.run}'] = cluster_labels

# Calculate silhouette score
output_file = "/home/oliven/scFoundationModels/notebooks/reprogramming/output_kmeans/silhouette_scores.txt"
score = silhouette_score(X_scaled, cluster_labels) if args.n_clusters > 1 else None
with open(output_file, "a") as f:  # Use "a" to append to the file
    f.write(f"Silhouette score for kmeans_{args.n_clusters}_{args.run}: {score:.4f}\n")

# Save the silhouette score in adata.uns
if 'silhouette_scores' not in adata.uns:
    adata.uns['silhouette_scores'] = {}
adata.uns['silhouette_scores'][f'kmeans_{args.n_clusters}_{args.run}'] = score

# Optionally save the updated adata object after each run
adata.write('/nfs/turbo/umms-indikar/shared/projects/DARPA_AI/in-silico-reprogramming/fibroblast_typing_9_20_n/output_kmeans/ts_fb_many_clusters.h5ad')
